In [4]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc for doc in docs)

a = {"question": "sdf", "documents": ["bgf", "fgsf", "fsfs"]}
tmp = itemgetter("question")
tmp1 = lambda input: format_docs(input["documents"])

In [5]:
tmp1(a)

'bgf\n\nfgsf\n\nfsfs'

In [2]:
tmp(a)

'sdf'

In [1]:
import os

In [6]:
os.path.join("./retrieval/", "routers.py")

'./retrieval/routers.py'

In [1]:
for k, v in {"a": "b"}.items():
    print(k)
    print(v)

a
b


In [1]:
from src.retrieval.utils import docs_chunking
from langchain.text_splitter import TokenTextSplitter

In [2]:
input_dir = "/home/vinhnq29/Public/advanced_RAG/langchain/db/wiki/docs"
extension = "text"
output_dir = "/home/vinhnq29/Public/advanced_RAG/langchain/db/wiki/chunks"
text_splitter = TokenTextSplitter(chunk_size=256, chunk_overlap=10)

def add_chunk_metadata(doc: dict, chunk_text: str) -> dict:
    chunk_metadata = {}
    for k, v in doc.items():
        chunk_metadata[f"doc_{k}"] = v
    chunk_metadata.pop("doc_text")
    return chunk_metadata

In [3]:
texts, metadatas = docs_chunking(
    input_dir=input_dir,
    extension=extension,
    output_dir=output_dir,
    text_splitter=text_splitter,
    add_chunk_metadata=add_chunk_metadata
)

In [ ]:
text